# Exploring baseballdatabank

In [188]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import glob
pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

DATABANK_DIR = "c:/users/matho/python/baseballdatabank"


I manually made the change to bailen01 in the github pull request.

The all-star file has one row per person per all-star game and includes people who were dropped for injury or other reasons. Also there are duplicates for the 2 AS game years in the late fifties.

In [24]:
allstar_df = pd.read_csv(DATABANK_DIR + "/core/allstarfull.csv")

In [25]:
allstar_df.head(5)

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,gomezle01,1933,0,ALS193307060,NYA,AL,1,1.0
1,ferreri01,1933,0,ALS193307060,BOS,AL,1,2.0
2,gehrilo01,1933,0,ALS193307060,NYA,AL,1,3.0
3,gehrich01,1933,0,ALS193307060,DET,AL,1,4.0
4,dykesji01,1933,0,ALS193307060,CHA,AL,1,5.0


In [62]:
def supplemental(df):
    xdf = df.copy()
    xdf['TB'] = xdf['H'] + xdf['2B'] + 2*xdf['3B'] + 3*xdf['HR']
    xdf['PA'] = xdf['AB'] + xdf['BB'] + xdf['HBP'] + xdf['SH'] + xdf['SF']

    xdf['AVG'] = xdf['H'] / xdf['AB']
    xdf['OBP'] = (xdf['H'] + xdf['BB'] + xdf['HBP']) / (xdf['PA'] - xdf['SH'])
    xdf['SLG'] = xdf['TB'] / xdf['AB']   
    return xdf
    

###  Baseruns accounting

In [206]:
batting_df = pd.read_csv(DATABANK_DIR + "/core/Batting.csv")
batting_df.columns = ['PLAYER_ID','YEAR','STINT','TEAM_ID','LEAGUE_ID','G','AB','R','HITS','2B','3B','HR','RBI','SB','CS','BB','SO','IBB','HP','SH','SF','GDP']
batting_df['TB'] = batting_df.HITS + batting_df['2B'] + 2*batting_df['3B'] + 3*batting_df['HR']
batting_df['PA'] = batting_df.AB + batting_df.BB + batting_df.SH + batting_df.SF + batting_df.HP

In [207]:
f = (batting_df.YEAR == 2006)

In [ ]:
lgdf['BASERUNS_A'] = lgdf.HITS + lgdf.BB + lgdf.HP - (0.5*lgdf.IBB) - lgdf.HR
lgdf['BASERUNS_B'] = 1.1*(1.4*lgdf.TB - 0.6*lgdf.HITS - 3*lgdf.HR + 0.1*(lgdf.BB + lgdf.HP - lgdf.IBB) + 0.9*(lgdf.SB - lgdf.CS - lgdf.GDP)) 
lgdf['BASERUNS_C'] = lgdf.PA - lgdf.BB - lgdf.SF - lgdf.SH - lgdf.HP - lgdf.HITS + lgdf.CS + lgdf.GDP
lgdf['BASERUNS_D'] = lgdf.HR
lgdf['BASERUNS'] = (lgdf.BASERUNS_A*lgdf.BASERUNS_B)/(lgdf.BASERUNS_B+lgdf.BASERUNS_C) + lgdf.BASERUNS_D


In [217]:
team_df = batting_df[f].groupby("TEAM_ID")[['R','PA','HITS','BB','IBB','HR','TB','HP','SB','CS','GDP','SH','SF']].sum()

team_df['BASERUNS_A'] = team_df.HITS + team_df.BB + team_df.HP - (0.5*team_df.IBB) - team_df.HR
team_df['BASERUNS_B'] = 1.1*(1.4*team_df.TB - 0.6*team_df.HITS - 3*team_df.HR + 0.1*(team_df.BB + team_df.HP - team_df.IBB) + 0.9*(team_df.SB - team_df.CS - team_df.GDP)) 
team_df['BASERUNS_C'] = team_df.PA - team_df.BB - team_df.SF - team_df.SH - team_df.HP - team_df.HITS + team_df.CS + team_df.GDP
team_df['BASERUNS_D'] = team_df.HR
team_df['BASERUNS'] = (team_df.BASERUNS_A*team_df.BASERUNS_B)/(team_df.BASERUNS_B+team_df.BASERUNS_C) + team_df.BASERUNS_D


In [216]:
player_df = batting_df[f].groupby(["PLAYER_ID","TEAM_ID"])[['R','PA','HITS','BB','IBB','HR','TB','HP','SB','CS','GDP','SH','SF']].sum()

player_df['BASERUNS_A'] = player_df.HITS + player_df.BB + player_df.HP - (0.5*player_df.IBB) - player_df.HR
player_df['BASERUNS_B'] = 1.1*(1.4*player_df.TB - 0.6*player_df.HITS - 3*player_df.HR + 0.1*(player_df.BB + player_df.HP - player_df.IBB) + 0.9*(player_df.SB - player_df.CS - player_df.GDP)) 
player_df['BASERUNS_C'] = player_df.PA - player_df.BB - player_df.SF - player_df.SH - player_df.HP - player_df.HITS + player_df.CS + player_df.GDP
player_df['BASERUNS_D'] = player_df.HR
player_df['BASERUNS'] = (player_df.BASERUNS_A*player_df.BASERUNS_B)/(player_df.BASERUNS_B+player_df.BASERUNS_C) + player_df.BASERUNS_D


In [221]:
pd.DataFrame({"player":player_df.groupby(['TEAM_ID']).BASERUNS.sum(),"team":team_df.BASERUNS})

,player,team
TEAM_ID,,
ARI,799.188604,787.107150
ATL,853.165121,837.608572
BAL,803.306269,799.690243
BOS,869.241917,861.464088
CHA,890.806261,883.660248
CHN,754.619155,740.330214
CIN,831.147952,820.121334
CLE,885.650699,881.161773
COL,834.593376,814.869714
